In [17]:
import pickle
import en_core_web_sm
import spacy
import random
from spacy import displacy

In [18]:
filename = '../training_data'
infile = open(filename,'rb')
TRAIN_DATA = pickle.load(infile)
infile.close()

In [19]:
nlp=spacy.load('en_core_web_sm')

In [20]:
def train_model(train_data):
    if 'ner' not in nlp.pipe_names:
        ner=nlp.create_pipe('ner')
        nlp.add_pipe(ner,last=True)
    else:
        ner = nlp.get_pipe("ner")
    for _,annotation in train_data:
        for ent in annotation['entities']:
            ner.add_label(ent[2])
    


    other_pipes = [pipe for pipe in nlp.pipe_names if pipe !='ner']
    # only train NER
    with nlp.disable_pipes(*other_pipes):
        optimizer=nlp.entity.create_optimizer()
        for itn in range(10):
            random.shuffle(train_data)
            losses = {}
            index=0
            for text,annotations in train_data:
                try:
                    nlp.update(
                        [text],  # batch of texts
                        [annotations],  # batch of annotations
                        drop=0.2,
                        sgd=optimizer,  # dropout - make it harder to memorise data
                        losses=losses)
                except Exception as e:
                    pass
            print(losses)
        
    

In [21]:
train_model(TRAIN_DATA)

{'ner': 1363.5893811372548}
{'ner': 1292.0212535316896}
{'ner': 1160.0665547975632}
{'ner': 1189.0017516318376}
{'ner': 1258.1401791689552}
{'ner': 1132.8750946703726}
{'ner': 1086.143523698841}
{'ner': 1096.4329109035023}
{'ner': 1010.4886286383512}
{'ner': 1084.25339865586}


In [22]:
nlp.to_disk('nlp_model')

In [23]:
nlp = spacy.load('nlp_model')
doc = nlp("Shareholders of Torrent Power will consider a proposal for a dividend of Rs 2.20 per share for 2016-17 at its annual general meeting on August 1")
for ent in doc.ents:
    print(ent.text,ent.label_)

annual general meeting CA_TYPE


In [15]:
displacy.serve(doc,style='ent')


Using the 'ent' visualizer
Serving on http://0.0.0.0:5000 ...

Shutting down server on port 5000.
